In [ ]:
import requests
import time

# ===== API KEY CONFIGURATION =====
# Get your API key from (you have to add your card to your account :p ): https://www.opendota.com/
# Leave as None if you don't have an API key (will use lower rate limits)
API_KEY = "aaa8d694-b754-4f74-b0d3-fbab49254269"  # Replace with your API key: "your-api-key-here"
# ==================================

match_ids = set()
target_ids = 200000   # Target: 20,0000 match IDs to ensure 10M+ float points (~500-1000 floats per match)
max_calls = 2000       # max # of API calls to /proMatches (100 matches per call = 20,000 total)
less_than_match_id = None  # Start with most recent matches

# Optimized for API key usage (faster with higher rate limits)
if API_KEY:
    base_delay = 0.5  # Delay between successful calls
    # Use separate connect and read timeouts for better control
    connect_timeout = 5  # How long to wait to establish connection
    read_timeout = 15    # How long to wait for response after connection
    timeout_retry_delay = 1.0  # Wait 1s between timeout retries
    max_retries = 1  # Only 1 retry for timeouts (fail fast, move on)
    max_consecutive_failures = 5  # More tolerance with API key
else:
    base_delay = 5  # Slower without API key
    connect_timeout = 10
    read_timeout = 20
    timeout_retry_delay = 2.0
    max_retries = 2
    max_consecutive_failures = 2

consecutive_failures = 0  # Track consecutive failures

for i in range(max_calls):
    print(f"Calling /proMatches ({i+1}/{max_calls})...")
    
    # Build URL with pagination parameter
    url = "https://api.opendota.com/api/proMatches"
    params = {}
    if less_than_match_id:
        params["less_than_match_id"] = less_than_match_id
    if API_KEY:
        params["api_key"] = API_KEY
    
    # Retry logic for 429 errors and timeouts
    retry_count = 0
    success = False
    
    while retry_count <= max_retries and not success:
        try:
            # Use tuple timeout: (connect_timeout, read_timeout)
            r = requests.get(url, params=params, timeout=(connect_timeout, read_timeout))
        except requests.exceptions.Timeout:
            # Handle timeout - retry once then move on (fail fast)
            retry_count += 1
            if retry_count <= max_retries:
                print(f"  ⏱️  Timeout after {connect_timeout + read_timeout}s. Retrying in {timeout_retry_delay:.1f}s... (attempt {retry_count}/{max_retries + 1})")
                time.sleep(timeout_retry_delay)
                continue
            else:
                print(f"  ❌ Timeout - moving to next call")
                break  # Stop retrying, move on
        except requests.exceptions.ConnectionError:
            # Connection errors - retry once
            retry_count += 1
            if retry_count <= max_retries:
                print(f"  🔌 Connection error. Retrying in {timeout_retry_delay:.1f}s...")
                time.sleep(timeout_retry_delay)
                continue
            else:
                print(f"  ❌ Connection failed - moving to next call")
                break
        except requests.exceptions.RequestException as e:
            # Other request exceptions
            print(f"  ⚠️  Request error: {type(e).__name__} - skipping this call")
            break  # Don't retry other errors
        
        if r.status_code == 429:
            # Rate limited - wait with exponential backoff
            retry_after = r.headers.get('Retry-After')
            if retry_after:
                wait_time = int(retry_after)
                # With API key, use shorter waits
                if API_KEY:
                    wait_time = max(wait_time, 10)  # Minimum 10s with API key
                else:
                    wait_time = max(wait_time, 60)  # Minimum 60s without API key
                print(f"  Rate limited! Waiting {wait_time} seconds (from Retry-After header)...")
            else:
                # Exponential backoff - faster with API key
                if API_KEY:
                    wait_time = 10 * (2 ** retry_count)  # 10s, 20s, 40s with API key
                else:
                    wait_time = 60 * (2 ** retry_count)  # 60s, 120s, 240s without API key
                print(f"  Rate limited! Waiting {wait_time} seconds (exponential backoff, attempt {retry_count + 1}/{max_retries + 1})...")
            
            time.sleep(wait_time)
            retry_count += 1
            continue
        
        if r.status_code != 200:
            print(f"  Got status {r.status_code} - skipping this call")
            break  # Break out of retry loop for non-429 errors
        
        # Success!
        success = True
        data = r.json()
        consecutive_failures = 0  # Reset failure counter on success
    
    # If we didn't get a successful response, skip this iteration
    if not success:
        consecutive_failures += 1
        print(f"  ⏭️  Skipping this call (consecutive failures: {consecutive_failures}/{max_consecutive_failures})")
        
        # If too many consecutive failures, stop to avoid wasting time
        if consecutive_failures >= max_consecutive_failures:
            print(f"\n⚠️  Stopping: {max_consecutive_failures} consecutive failures.")
            print(f"   Collected {len(match_ids)} match IDs so far.")
            print("   RECOMMENDATIONS:")
            print("   1. Wait 10-15 minutes before trying again")
            print("   2. Check if API is responding: https://api.opendota.com/api/health")
            print("   3. Run this cell again later")
            break
        
        # Minimal delay after failure - just move on quickly
        time.sleep(0.1)  # Very short delay
        continue
    
    # If we got no data or same matches, break
    if not data:
        print("  No more matches available")
        break
    
    before = len(match_ids)
    oldest_match_id = None
    
    for m in data:
        mid = m.get("match_id")
        if mid:
            match_ids.add(mid)
            # Track the oldest (smallest) match_id for pagination
            if oldest_match_id is None or mid < oldest_match_id:
                oldest_match_id = mid
    
    after = len(match_ids)
    print(f"  Unique IDs so far: {after} (+{after - before} new)")

    if len(match_ids) >= target_ids:
        break
    
    # Update pagination parameter to get older matches
    if oldest_match_id:
        less_than_match_id = oldest_match_id
    else:
        break  # No new matches found
    
    time.sleep(base_delay)  # be polite to the API

match_ids = list(match_ids)
print("\nTotal unique match IDs collected:", len(match_ids))
print("First 10 IDs:", match_ids[:10])

# Save match_ids to file for recovery if kernel restarts
import os
import pickle
os.makedirs("checkpoints", exist_ok=True)
with open("checkpoints/match_ids.pkl", "wb") as f:
    pickle.dump(match_ids, f)
print(f"💾 Saved {len(match_ids)} match IDs to checkpoints/match_ids.pkl")


Calling /proMatches (1/2000)...
  Unique IDs so far: 100 (+100 new)
Calling /proMatches (2/2000)...
  Unique IDs so far: 200 (+100 new)
Calling /proMatches (3/2000)...
  Unique IDs so far: 300 (+100 new)
Calling /proMatches (4/2000)...
  Unique IDs so far: 400 (+100 new)
Calling /proMatches (5/2000)...
  Unique IDs so far: 500 (+100 new)
Calling /proMatches (6/2000)...
  Unique IDs so far: 600 (+100 new)
Calling /proMatches (7/2000)...
  Unique IDs so far: 700 (+100 new)
Calling /proMatches (8/2000)...
  Unique IDs so far: 800 (+100 new)
Calling /proMatches (9/2000)...
  Unique IDs so far: 900 (+100 new)
Calling /proMatches (10/2000)...
  Unique IDs so far: 1000 (+100 new)
Calling /proMatches (11/2000)...
  Unique IDs so far: 1100 (+100 new)
Calling /proMatches (12/2000)...
  Unique IDs so far: 1200 (+100 new)
Calling /proMatches (13/2000)...
  Unique IDs so far: 1300 (+100 new)
Calling /proMatches (14/2000)...
  Unique IDs so far: 1400 (+100 new)
Calling /proMatches (15/2000)...
  Uni

In [3]:
import pandas as pd
import numpy as np

class DataPreprocessing:
    """Class to extract and organize Dota 2 match data from OpenDota API responses."""
    
    def __init__(self):
        # Use lists for memory efficiency - convert to DataFrame only when needed
        self.matches_list = []
        self.players_list = []
        self.objectives_list = []
        self.advantages_list = []
        self.events_list = []
        self.abilities_list = []
        self.wards_list = []
        
        # Cache for DataFrames (lazy loading)
        self._matches_df = None
        self._players_df = None
        self._objectives_df = None
        self._advantages_df = None
        self._events_df = None
        self._abilities_df = None
        self._wards_df = None

    def get_match(self, match):
        """Get general information from the match and append to self.matches_list."""
        fields = ['match_id', 'match_seq_num', 'patch', 'region', 'start_time', 'duration',
                  'game_mode', 'skill', 'first_blood_time', 'barracks_status_dire',
                  'barracks_status_radiant', 'tower_status_dire', 'tower_status_radiant',
                  'dire_score', 'radiant_score', 'radiant_win']
        proc_match = {key: match.get(key) for key in fields}
        self.matches_list.append(proc_match)
        self._matches_df = None  # Invalidate cache

    def get_match_objectives(self, match):
        """Get game objectives like Roshan and towers and append to self.objectives_list."""
        fields = ['time', 'type', 'unit', 'key', 'slot', 'player_slot']
        if match.get('objectives'):
            for item in match['objectives']:
                obj = {'match_id': match['match_id']}
                for field in fields:
                    obj[field] = item.get(field, np.nan)
                self.objectives_list.append(obj)
            self._objectives_df = None  # Invalidate cache

    def get_match_advantages(self, match):
        """Get radiant gold and xp advantage for each minute and append to self.advantages_list."""
        if match.get('radiant_gold_adv'):  # Gold advantage (gold_or_xp = 0)
            for i, value in enumerate(match['radiant_gold_adv']):
                self.advantages_list.append({
                    'match_id': match['match_id'],
                    'minute': i,
                    'gold_or_xp': 0,
                    'value': int(value)
                })
        if match.get('radiant_xp_adv'):  # XP advantage (gold_or_xp = 1)
            for i, value in enumerate(match['radiant_xp_adv']):
                self.advantages_list.append({
                    'match_id': match['match_id'],
                    'minute': i,
                    'gold_or_xp': 1,
                    'value': int(value)
                })
        if match.get('radiant_gold_adv') or match.get('radiant_xp_adv'):
            self._advantages_df = None  # Invalidate cache

    def get_players_events(self, match):
        """Get events for each player (kills, runes, bb and purchases) and append to self.events_list."""
        for player in match.get('players', []):
            if player.get('buyback_log'):  # Player's Buybacks
                for bb in player['buyback_log']:
                    self.events_list.append({
                        'match_id': match['match_id'],
                        'account_id': player.get('account_id'),
                        'player_slot': player.get('player_slot'),
                        'hero_id': player.get('hero_id'),
                        'time': bb.get('time'),
                        'key': np.nan,
                        'event': 'buyback'
                    })
            if player.get('kills_log'):  # Player's kills on enemy heroes
                for kill in player['kills_log']:
                    self.events_list.append({
                        'match_id': match['match_id'],
                        'account_id': player.get('account_id'),
                        'player_slot': player.get('player_slot'),
                        'hero_id': player.get('hero_id'),
                        'time': kill.get('time'),
                        'key': kill.get('key'),
                        'event': 'kill'
                    })
            if player.get('runes_log'):  # Runes picked
                for rune in player['runes_log']:
                    self.events_list.append({
                        'match_id': match['match_id'],
                        'account_id': player.get('account_id'),
                        'player_slot': player.get('player_slot'),
                        'hero_id': player.get('hero_id'),
                        'time': rune.get('time'),
                        'key': rune.get('key'),
                        'event': 'rune'
                    })
            if player.get('purchase_log'):
                for item in player['purchase_log']:  # Items purchased
                    self.events_list.append({
                        'match_id': match['match_id'],
                        'account_id': player.get('account_id'),
                        'player_slot': player.get('player_slot'),
                        'hero_id': player.get('hero_id'),
                        'time': item.get('time'),
                        'key': item.get('key'),
                        'event': 'purchase'
                    })
        if any(player.get(key) for player in match.get('players', []) 
               for key in ['buyback_log', 'kills_log', 'runes_log', 'purchase_log']):
            self._events_df = None  # Invalidate cache

    def get_ability_upgrades(self, match):
        """Get skill upgrades for each player. Columns goes from 1 to 25 for each possible skill upgrade."""
        for player in match.get('players', []):
            if player.get('ability_upgrades_arr'):
                tmp = {
                    'match_id': match['match_id'],
                    'account_id': player.get('account_id'),
                    'player_slot': player.get('player_slot'),
                    'hero_id': player.get('hero_id'),
                }
                for i in range(25):
                    tmp['skill_upgrade_' + str(i + 1)] = np.nan
                for i, value in enumerate(player['ability_upgrades_arr']):
                    if i < 25:
                        tmp['skill_upgrade_' + str(i + 1)] = value
                self.abilities_list.append(tmp)
        if any(player.get('ability_upgrades_arr') for player in match.get('players', [])):
            self._abilities_df = None  # Invalidate cache

    def get_wards(self, match):
        """Get time, position, slot and hero for each ward placed and append to self.wards_list."""
        for player in match.get('players', []):
            if player.get('obs_log'):  # Observer wards (type = 0)
                for item in player['obs_log']:
                    self.wards_list.append({
                        'match_id': match['match_id'],
                        'account_id': player.get('account_id'),
                        'player_slot': player.get('player_slot'),
                        'hero_id': player.get('hero_id'),
                        'time': item.get('time'),
                        'x': item.get('x'),
                        'y': item.get('y'),
                        'type': 0
                    })
            if player.get('sen_log'):  # Sentry wards (type = 1)
                for item in player['sen_log']:
                    self.wards_list.append({
                        'match_id': match['match_id'],
                        'account_id': player.get('account_id'),
                        'player_slot': player.get('player_slot'),
                        'hero_id': player.get('hero_id'),
                        'time': item.get('time'),
                        'x': item.get('x'),
                        'y': item.get('y'),
                        'type': 1
                    })
        if any(player.get(key) for player in match.get('players', []) 
               for key in ['obs_log', 'sen_log']):
            self._wards_df = None  # Invalidate cache

    def get_players(self, match):
        """Get match information for each player and append to self.players_list."""
        fields = ['player_slot', 'account_id', 'hero_id', 'kills', 'deaths',
                  'assists', 'last_hits', 'denies', 'gold_per_min', 'xp_per_min',
                  'gold_spent', 'hero_damage', 'hero_healing', 'tower_damage',
                  'level', 'party_size', 'item_0', 'item_1', 'item_2', 'item_3',
                  'item_4', 'item_5', 'camps_stacked', 'creeps_stacked', 'obs_placed', 'sen_placed',
                  'purchase_tpscroll', 'rune_pickups', 'roshans_killed', 'towers_killed', 'win']
        for item in match.get('players', []):
            player = {'match_id': match['match_id']}
            for field in fields:
                player[field] = item.get(field, np.nan)
            self.players_list.append(player)
        if match.get('players'):
            self._players_df = None  # Invalidate cache

    def get_all_current_match_tables(self, match_details):
        """Get all tables from a current match."""
        self.get_match(match_details)
        self.get_players(match_details)
        self.get_match_objectives(match_details)
        self.get_match_advantages(match_details)
        self.get_ability_upgrades(match_details)
        self.get_players_events(match_details)
        self.get_wards(match_details)
    
    def clear_dataframe_cache(self):
        """Clear cached DataFrames to free memory. Lists remain intact."""
        self._matches_df = None
        self._players_df = None
        self._objectives_df = None
        self._advantages_df = None
        self._events_df = None
        self._abilities_df = None
        self._wards_df = None
    
    # Properties for backward compatibility - lazy DataFrame conversion
    @property
    def matches(self):
        if self._matches_df is None:
            self._matches_df = pd.DataFrame(self.matches_list) if self.matches_list else pd.DataFrame()
        return self._matches_df
    
    @property
    def players(self):
        if self._players_df is None:
            self._players_df = pd.DataFrame(self.players_list) if self.players_list else pd.DataFrame()
        return self._players_df
    
    @property
    def objectives(self):
        if self._objectives_df is None:
            self._objectives_df = pd.DataFrame(self.objectives_list) if self.objectives_list else pd.DataFrame()
        return self._objectives_df
    
    @property
    def advantages(self):
        if self._advantages_df is None:
            self._advantages_df = pd.DataFrame(self.advantages_list) if self.advantages_list else pd.DataFrame()
        return self._advantages_df
    
    @property
    def events(self):
        if self._events_df is None:
            self._events_df = pd.DataFrame(self.events_list) if self.events_list else pd.DataFrame()
        return self._events_df
    
    @property
    def abilities(self):
        if self._abilities_df is None:
            self._abilities_df = pd.DataFrame(self.abilities_list) if self.abilities_list else pd.DataFrame()
        return self._abilities_df
    
    @property
    def wards(self):
        if self._wards_df is None:
            self._wards_df = pd.DataFrame(self.wards_list) if self.wards_list else pd.DataFrame()
        return self._wards_df

# Initialize the data preprocessor
preprocessor = DataPreprocessing()


In [ ]:
import requests
import time
import os
import pickle
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from threading import Lock

# ===== API KEY CONFIGURATION =====
# Get your free API key from: https://www.opendota.com/
# Leave as None if you don't have an API key (will use lower rate limits)
# IMPORTANT: Use the SAME API key as in Cell 0!
API_KEY = "aaa8d694-b754-4f74-b0d3-fbab49254269"  # Replace with your API key: "your-api-key-here"
# ==================================

# Thread-safe counters
success = 0
target_success = 150000   # Target: 150,000 matches to ensure 10M+ float points (~500-1000 floats per match)
save_interval = 5000  # Save progress every 5000 matches to prevent data loss (more frequent for large datasets)
csv_save_interval = 10000  # Save incremental CSV files every 10k matches
skipped_status = 0
skipped_players = 0
errors = 0
counter_lock = Lock()

# Optimized for API key usage (faster with higher rate limits)
if API_KEY:
    base_delay = 0.1  # Very fast with API key
    max_retries = 2
    max_workers = 10  # More concurrent requests with API key
else:
    base_delay = 0.3  # Slower without API key
    max_retries = 2
    max_workers = 5  # Fewer concurrent requests without API key

def fetch_match(mid, idx, total):
    """Fetch a single match with retry logic."""
    global success, skipped_status, skipped_players, errors
    
    retry_count = 0
    request_success = False
    match = None
    
    while retry_count <= max_retries and not request_success:
        try:
            params = {}
            if API_KEY:
                params['api_key'] = API_KEY
            r = requests.get(
                f"https://api.opendota.com/api/matches/{mid}",
                params=params,
                timeout=5
            )

            if r.status_code == 429:
                # Rate limited - wait with exponential backoff (faster with API key)
                retry_after = r.headers.get('Retry-After')
                if retry_after:
                    wait_time = int(retry_after)
                    if API_KEY:
                        wait_time = max(wait_time, 5)  # Minimum 5s with API key
                    else:
                        wait_time = max(wait_time, 10)  # Minimum 10s without API key
                else:
                    # Exponential backoff - faster with API key
                    if API_KEY:
                        wait_time = (2 ** retry_count) * base_delay * 2  # Very short with API key
                    else:
                        wait_time = (2 ** retry_count) * base_delay * 4  # Longer without API key
                
                time.sleep(wait_time)
                retry_count += 1
                continue

            if r.status_code != 200:
                with counter_lock:
                    skipped_status += 1
                return None  # Skip this match

            # Success - parse the match data
            match = r.json()
            request_success = True

        except requests.exceptions.Timeout:
            with counter_lock:
                errors += 1
            return None
        except Exception as e:
            with counter_lock:
                errors += 1
            return None
    
    if not request_success or match is None:
        return None

    # Validate players
    players = match.get("players")
    if not players or len(players) != 10:
        with counter_lock:
            skipped_players += 1
        return None

    # Return match data for processing
    return match

# Function to save progress periodically
def save_progress(checkpoint_dir="checkpoints", save_csv=False):
    """Save preprocessor state to prevent data loss."""
    os.makedirs(checkpoint_dir, exist_ok=True)
    checkpoint_file = os.path.join(checkpoint_dir, "preprocessor_checkpoint.pkl")
    
    # Save the lists (much smaller than DataFrames)
    checkpoint_data = {
        'matches_list': preprocessor.matches_list,
        'players_list': preprocessor.players_list,
        'objectives_list': preprocessor.objectives_list,
        'advantages_list': preprocessor.advantages_list,
        'events_list': preprocessor.events_list,
        'abilities_list': preprocessor.abilities_list,
        'wards_list': preprocessor.wards_list,
    }
    
    with open(checkpoint_file, 'wb') as f:
        pickle.dump(checkpoint_data, f)
    print(f"  💾 Checkpoint saved: {len(preprocessor.matches_list)} matches")
    
    # Optionally save incremental CSV files
    if save_csv:
        csv_dir = os.path.join(checkpoint_dir, "incremental_csv")
        os.makedirs(csv_dir, exist_ok=True)
        match_count = len(preprocessor.matches_list)
        
        # Save incremental CSVs
        pd.DataFrame(preprocessor.matches_list).to_csv(
            os.path.join(csv_dir, f"matches_{match_count}.csv"), index=False
        )
        pd.DataFrame(preprocessor.players_list).to_csv(
            os.path.join(csv_dir, f"players_{match_count}.csv"), index=False
        )
        print(f"  📄 Incremental CSVs saved at {match_count} matches")
    
    # Clear DataFrame cache to free memory
    preprocessor.clear_dataframe_cache()

# Check if match_ids exists - if not, try to load from saved file
if 'match_ids' not in globals() or match_ids is None or len(match_ids) == 0:
    # Try to load from saved match_ids.pkl file (from Cell 0)
    match_ids_file = "checkpoints/match_ids.pkl"
    if os.path.exists(match_ids_file):
        print(f"⚠️  match_ids not found in memory! Loading from saved file: {match_ids_file}")
        with open(match_ids_file, 'rb') as f:
            match_ids = pickle.load(f)
        print(f"✅ Loaded {len(match_ids)} match IDs from saved file")
    else:
        # Try loading from CSV checkpoints as fallback
        import glob
        csv_files = glob.glob("checkpoints/incremental_csv/matches_*.csv")
        if csv_files:
            latest_csv = max(csv_files, key=os.path.getmtime)
            print(f"⚠️  match_ids not found! Loading from latest checkpoint: {latest_csv}")
            match_ids_df = pd.read_csv(latest_csv)
            match_ids = match_ids_df['match_id'].tolist()
            print(f"✅ Loaded {len(match_ids)} match IDs from checkpoint CSV")
        else:
            raise NameError(
                "❌ ERROR: 'match_ids' is not defined!\n\n"
                "You need to run Cell 0 first to collect match IDs from the API.\n"
                "Cell 0 will save match_ids to checkpoints/match_ids.pkl\n\n"
                "Please run Cell 0 to fetch match IDs, then run this cell again."
            )

# Process matches concurrently
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    # Submit enough tasks (submit more than target to account for failures)
    tasks_to_submit = min(len(match_ids), target_success * 2)
    future_to_match = {}
    
    for idx in range(tasks_to_submit):
        mid = match_ids[idx]
        future = executor.submit(fetch_match, mid, idx, len(match_ids))
        future_to_match[future] = (idx, mid)
    
    # Process completed tasks
    for future in as_completed(future_to_match):
        with counter_lock:
            current_success = success
        if current_success >= target_success:
            break
            
        idx, mid = future_to_match[future]
        try:
            match = future.result()
            
            if match is None:
                continue
            
            # Process the match data
            try:
                preprocessor.get_all_current_match_tables(match)
                with counter_lock:
                    success += 1
                    current_success = success
                
                # Print progress every 100 matches or at milestones
                if current_success % 100 == 0 or current_success in [1, 10, 50, 100, 500, 1000, 5000, 10000, 50000, 100000]:
                    progress_pct = (current_success / target_success) * 100
                    print(f"  ✅ Match {idx+1} (ID: {mid}) - Progress: {current_success:,}/{target_success:,} ({progress_pct:.1f}%)")
                elif current_success <= 10:
                    print(f"  ✅ Match {idx+1} (ID: {mid}) - Success count: {current_success}")
                
                # Save checkpoint periodically to prevent data loss
                if current_success % save_interval == 0:
                    save_csv = (current_success % csv_save_interval == 0)
                    save_progress(save_csv=save_csv)
                
                # Clear memory cache every 1000 matches to prevent memory buildup
                if current_success % 1000 == 0:
                    preprocessor.clear_dataframe_cache()
            except Exception as e:
                with counter_lock:
                    errors += 1
                print(f"  Error processing match {idx+1}: {e}")
                
        except Exception as e:
            with counter_lock:
                errors += 1
            print(f"  Exception for match {idx+1}: {e}")
        
        # Small delay to avoid overwhelming the API
        time.sleep(base_delay / max_workers)

print("\n=== Summary ===")
print("Successful matches:", success)
print("Skipped (status != 200):", skipped_status)
print("Skipped (bad players):", skipped_players)
print("Errors/Timeouts:", errors)

print("\n=== DataFrames Created ===")
print(f"Matches: {len(preprocessor.matches)} rows")
print(f"Players: {len(preprocessor.players)} rows")
print(f"Objectives: {len(preprocessor.objectives)} rows")
print(f"Advantages: {len(preprocessor.advantages)} rows")
print(f"Events: {len(preprocessor.events)} rows")
print(f"Abilities: {len(preprocessor.abilities)} rows")
print(f"Wards: {len(preprocessor.wards)} rows")

# Calculate total float points
total_floats = 0
for df_name, df in [
    ('matches', preprocessor.matches),
    ('players', preprocessor.players),
    ('objectives', preprocessor.objectives),
    ('advantages', preprocessor.advantages),
    ('events', preprocessor.events),
    ('abilities', preprocessor.abilities),
    ('wards', preprocessor.wards)
]:
    if len(df) > 0:
        # Count numeric columns (float/int types)
        numeric_cols = df.select_dtypes(include=['float64', 'int64', 'float32', 'int32']).shape[1]
        floats_in_df = len(df) * numeric_cols
        total_floats += floats_in_df
        print(f"  {df_name}: ~{floats_in_df:,} floats ({len(df)} rows × {numeric_cols} numeric cols)")

print(f"\n{'='*50}")
print(f"📊 TOTAL FLOAT POINTS: {total_floats:,}")
print(f"   Target: 10,000,000")
if total_floats >= 10_000_000:
    print(f"   ✅ REQUIREMENT MET! ({total_floats/10_000_000:.2f}x target)")
else:
    print(f"   ⚠️  Need {10_000_000 - total_floats:,} more floats ({((10_000_000 - total_floats) / total_floats * len(preprocessor.matches)):.0f} more matches estimated)")
print(f"{'='*50}")

# Display the matches dataframe as the main output
print("\n=== Matches DataFrame ===")
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
preprocessor.matches


⚠️  match_ids not found! Loading from latest checkpoint: checkpoints/incremental_csv/matches_70000.csv
✅ Loaded 70000 match IDs from checkpoint CSV
  Error processing match 1: name 'preprocessor' is not defined
  Error processing match 5: name 'preprocessor' is not defined
  Error processing match 3: name 'preprocessor' is not defined
  Error processing match 4: name 'preprocessor' is not defined
  Error processing match 7: name 'preprocessor' is not defined
  Error processing match 6: name 'preprocessor' is not defined
  Error processing match 9: name 'preprocessor' is not defined
  Error processing match 8: name 'preprocessor' is not defined
  Error processing match 11: name 'preprocessor' is not defined
  Error processing match 12: name 'preprocessor' is not defined
  Error processing match 14: name 'preprocessor' is not defined
  Error processing match 13: name 'preprocessor' is not defined
  Error processing match 10: name 'preprocessor' is not defined
  Error processing match 17:

In [ ]:
import os

# Create Data_Individual folder if it doesn't exist
os.makedirs("Data_Individual", exist_ok=True)

# Save all dataframes to CSV files in Data_Individual folder
preprocessor.matches.to_csv("Data_Individual/dota_pro_matches.csv", index=False)
preprocessor.players.to_csv("Data_Individual/dota_pro_players.csv", index=False)
preprocessor.objectives.to_csv("Data_Individual/dota_pro_objectives.csv", index=False)
preprocessor.advantages.to_csv("Data_Individual/dota_pro_advantages.csv", index=False)
preprocessor.events.to_csv("Data_Individual/dota_pro_events.csv", index=False)
preprocessor.abilities.to_csv("Data_Individual/dota_pro_abilities.csv", index=False)
preprocessor.wards.to_csv("Data_Individual/dota_pro_wards.csv", index=False)

print("✅ All individual dataframes saved to Data_Individual folder!")


All dataframes saved to CSV files!


In [ ]:
# Combine all dataframes into one comprehensive CSV file
# Start with matches as the base
combined_df = preprocessor.matches.copy()

# Aggregate player statistics by team
if len(preprocessor.players) > 0:
    # Separate radiant and dire players
    players_df = preprocessor.players.copy()
    players_df['is_radiant'] = players_df['player_slot'] < 5
    
    # Aggregate by match and team
    player_agg = players_df.groupby(['match_id', 'is_radiant']).agg({
        'kills': 'sum',
        'deaths': 'sum',
        'assists': 'sum',
        'last_hits': 'sum',
        'denies': 'sum',
        'gold_per_min': 'sum',
        'xp_per_min': 'sum',
        'gold_spent': 'sum',
        'hero_damage': 'sum',
        'hero_healing': 'sum',
        'tower_damage': 'sum',
        'level': 'mean',
        'obs_placed': 'sum',
        'sen_placed': 'sum',
        'roshans_killed': 'sum',
        'towers_killed': 'sum'
    }).reset_index()
    
    # Pivot to get radiant and dire columns
    player_pivot = player_agg.pivot(index='match_id', columns='is_radiant', 
                                     values=['kills', 'deaths', 'assists', 'last_hits', 'denies',
                                            'gold_per_min', 'xp_per_min', 'gold_spent', 
                                            'hero_damage', 'hero_healing', 'tower_damage',
                                            'level', 'obs_placed', 'sen_placed', 
                                            'roshans_killed', 'towers_killed'])
    
    # Flatten column names
    player_pivot.columns = [f"{col[0]}_{'radiant' if col[1] else 'dire'}" for col in player_pivot.columns]
    player_pivot = player_pivot.reset_index()
    
    # Merge with combined_df
    combined_df = combined_df.merge(player_pivot, on='match_id', how='left')

# Aggregate objectives (count by type)
if len(preprocessor.objectives) > 0:
    obj_counts = preprocessor.objectives.groupby(['match_id', 'type']).size().reset_index(name='count')
    obj_pivot = obj_counts.pivot(index='match_id', columns='type', values='count').fillna(0)
    obj_pivot.columns = [f"objective_type_{col}" for col in obj_pivot.columns]
    obj_pivot = obj_pivot.reset_index()
    combined_df = combined_df.merge(obj_pivot, on='match_id', how='left')

# Aggregate advantages (get final advantage values)
if len(preprocessor.advantages) > 0:
    # Get final gold and XP advantage
    final_adv = preprocessor.advantages.groupby(['match_id', 'gold_or_xp']).last().reset_index()
    final_adv_pivot = final_adv.pivot(index='match_id', columns='gold_or_xp', values='value')
    final_adv_pivot.columns = ['final_gold_advantage', 'final_xp_advantage']
    final_adv_pivot = final_adv_pivot.reset_index()
    combined_df = combined_df.merge(final_adv_pivot, on='match_id', how='left')

# Aggregate events (count by event type)
if len(preprocessor.events) > 0:
    event_counts = preprocessor.events.groupby(['match_id', 'event']).size().reset_index(name='count')
    event_pivot = event_counts.pivot(index='match_id', columns='event', values='count').fillna(0)
    event_pivot.columns = [f"event_{col}_count" for col in event_pivot.columns]
    event_pivot = event_pivot.reset_index()
    combined_df = combined_df.merge(event_pivot, on='match_id', how='left')

# Aggregate wards (count by type)
if len(preprocessor.wards) > 0:
    ward_counts = preprocessor.wards.groupby(['match_id', 'type']).size().reset_index(name='count')
    ward_pivot = ward_counts.pivot(index='match_id', columns='type', values='count').fillna(0)
    ward_pivot.columns = ['observer_wards_placed', 'sentry_wards_placed']
    ward_pivot = ward_pivot.reset_index()
    combined_df = combined_df.merge(ward_pivot, on='match_id', how='left')

# Aggregate abilities (count skill upgrades per match)
if len(preprocessor.abilities) > 0:
    # Count non-null skill upgrades per match
    abilities_df = preprocessor.abilities.copy()
    skill_cols = [col for col in abilities_df.columns if col.startswith('skill_upgrade_')]
    abilities_df['total_skill_upgrades'] = abilities_df[skill_cols].notna().sum(axis=1)
    abilities_agg = abilities_df.groupby('match_id')['total_skill_upgrades'].sum().reset_index()
    abilities_agg.columns = ['match_id', 'total_skill_upgrades']
    combined_df = combined_df.merge(abilities_agg, on='match_id', how='left')

# Create Data_Combined folder if it doesn't exist
import os
os.makedirs("Data_Combined", exist_ok=True)

# Save combined dataframe to Data_Combined folder
combined_df.to_csv("Data_Combined/dota_pro_combined.csv", index=False)
print(f"\n✅ Combined CSV saved to Data_Combined folder! Shape: {combined_df.shape}")
print(f"Columns: {len(combined_df.columns)}")
print(f"\nFirst few columns: {list(combined_df.columns[:10])}...")



✅ Combined CSV saved! Shape: (15000, 65)
Columns: 65

First few columns: ['match_id', 'match_seq_num', 'patch', 'region', 'start_time', 'duration', 'game_mode', 'skill', 'first_blood_time', 'barracks_status_dire']...
